# Preprocessing and Quality Control of LC-MS Data with the nPYc-Toolbox

This tutorial demonstrates how to use the LC-MS data processing modules of the nPYc-Toolbox, to import and perform some basic preprocessing and quality control of LC-MS data, and to output a final high quality dataset ready for data modeling.

Details of how to install all of the required dependencies and to set up your computing environment can be found in 'document.txt', and full documentation for the nPYc-Toolbox can be found on [read the docs](https://npyc-toolbox.readthedocs.io/en/latest/index.html)

*****TODO - Create document.txt - or maybe we can just link to the read the docs page?*****

The nPYc-Toolbox has been developed based on the quality control criteria previously described in [Lewis et al. 2016](https://www.ncbi.nlm.nih.gov/pubmed/27479709)

The dataset used in this example (DEVSET U RPOS xcms) is comprised of six samples of pooled human urine, aliquoted, and independently prepared and measured by ultra-performance liquid chromatography coupled to reversed-phase spectrometry (LC-MS). Each source sample was separately prepared and assayed thirteen times. A pooled QC sample (study reference, SR) and independent external reference (long-term reference, LTR) of a comparable matrix was also acquired to assist in assessing analytical precision. See the Metabolights Study [MTBLS694](https://www.ebi.ac.uk/metabolights/MTBLS694) for details.

# 1. Import the nPYc-Toolbox and Configure the Jupyter Notebook

*****TODO - delete next two cells once nPYc updated*****

In [2]:
toolboxPath = '/Users/cs401/Box Sync/Carolines code/phenomecentre/npyc-toolbox-master'

In [3]:
import sys
import pyChemometrics
sys.path.append(toolboxPath)

In [4]:
# Import the nPYc-Toolbox
import nPYc

# Import enumerations for sample type
from nPYc.enumerations import VariableType, DatasetLevel, AssayRole, SampleType

# Import normalisation objects for data normalisation
from nPYc.utilities.normalisation import NullNormaliser, TotalAreaNormaliser, ProbabilisticQuotientNormaliser

# Import matplotlib plotting, configure the Jupyter notebook to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

# Set up plotly to work in offline mode with the notebook
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# 2. Import LC-MS Data

The first step is to import the LC-MS data into an nPYc-Toolbox [MSDataset object](https://npyc-toolbox.readthedocs.io/en/latest/objects.html).

The nPYc-Toolbox works on feature extracted LC-MS data, where the raw analytical data has been peak-picked into a one dimensional list of detected features, each of which is characterised by abundance and observed m/z and retention time. There are a number of peak-detection algorithms [Spicer et al. 2017](https://www.ncbi.nlm.nih.gov/pubmed/28890673), many of which are supported by the nPYc-Toolbox, however in this example, untargeted feature extraction was carried out using XCMS [Smith et al. 2006](https://www.ncbi.nlm.nih.gov/pubmed/16448051).

In brief, the raw data files were first converted from .RAW format to .mzML format using the msconvert tool from the ProteoWizard tookit [Chambers et al. 2012](https://www.ncbi.nlm.nih.gov/pubmed/23051804), for feature-detection with XCMS. msconvert was configured to discard scans below an intensity threshold of 100 (--filter 'threshold absolute 100 most-intense'), and only retain scans from the first acquisition function (--filter 'scanEvent 1'). Subsequent peak detection by XCMS used the centwave algorithm configured with a noise threshold of 600, mass accuracy of 25ppm, and minmum and maximum peakwidth of 1.5 and 5 seconds. Retention time alignment was performed using the 'density' method.

The LC-MS XCMS DevSet Dataset is located in 'DEVSET U RPOS xcms.csv'.

The following line creates an object representing the dataset.

In [ ]:
msData = nPYc.MSDataset('DEVSET U RPOS xcms.csv', fileType='XCMS', sop='GenericMS')

The “sop” parameter points to a configuration file (encoded in JSON format) which contains a set of parameters to use during data import and pre-processing, see [Configuration Files](https://npyc-toolbox.readthedocs.io/en/latest/configuration/configuration.html) for full details.

The default configuration file for LC-MS data, 'GenericMS', contains the recommended parameters for import and quality control of both human urine and plasma/serum data. For a list of all the parameters for MS data, see Table 5 in [Built-in Configuration SOPs](https://npyc-toolbox.readthedocs.io/en/latest/configuration/Built-in-Configuration-SOPs.html).

If required, users can create new configuration files, or indeed amend the existing documents with their own values, (see [Configuration Files](https://npyc-toolbox.readthedocs.io/en/latest/configuration/configuration.html)), however, any of the parameters present in these files can also be overwritten by passing values into the data import command directly, without having to modify or generate the configuration files themselves.

For LC-MS data, the majority of the parameters relate to preprocessing at the feature filtering stage, and therefore discussed in more detail below, but as an example, to change the threshold for filtering features based on residual standard deviation (RSD), the argument "rsdThreshold" can be overridden in the following manner:

```
msData = nPYc.MSDataset('DEVSET U RPOS xcms.csv', fileType='XCMS', sop='GenericMS', rsdThreshold=20)
```

Each nPYc Dataset object contains a name that can be changed as shown in the next cell. This name will be used in the summary and visualisation reports and in the file names of the exported data.

In [ ]:
msData.name = 'nPYc LC-MS Tutorial dataset'

# 3. Import Sample Metadata and Match to Acquired Data

The default way to add sample metadata is to prepare a CSV file which follows the set of conventions as described in [Sample Metadata](https://npyc-toolbox.readthedocs.io/en/latest/samplemetadata.html) and match it with the acquired data using the '.addSampleInfo' method.

Although optional, this is recommended in order to make optimal use of the quality control features and visualisations provided by the nPYc-Toolbox.

An example CSV file is provided, as given in 'DEVSET U RPOS Basic CSV.csv':

In [ ]:
msData.addSampleInfo(descriptionFormat='Basic CSV', filePath='DEVSET U RPOS Basic CSV')

As described in [Datasets](https://npyc-toolbox.readthedocs.io/en/latest/objects.html), the spectral data, sample metadata and feature metadata can be inspected directly using:

```
dataset.intensityData
dataset.sampleMetadata
dataset.featureMetadata

```



In [ ]:
# dataset.addSampleInfo(descriptionFormat='Raw Data', filePath=rawDataPath)

### Exclude features outside of the useful retention time range of the assay

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        Use the 'excludeFeatures' function to mark features outside of the useful RT range for exclusion. For the RPOS assay this corresponds to features with a retention time outside 0.6-10.5. 
        <br/><br/>
        Subsequently, use the 'applyMasks' function to permanently remove these features from the dataset.
    </font>
</div>

In [ ]:
print('Number of original features: ' + str(sum(dataset.featureMask)))
dataset.excludeFeatures(dataset.featureMetadata[dataset.featureMetadata['Retention Time'] > 10.5]['Feature Name'], on='Feature Name', message='Outside RT limits')
dataset.excludeFeatures(dataset.featureMetadata[dataset.featureMetadata['Retention Time'] < 0.6]['Feature Name'], on='Feature Name', message='Outside RT limits')
print('Number of features within RT range: ' + str(sum(dataset.featureMask)))

In [ ]:
dataset.applyMasks()

# 3. Sample & Feature Filtering

### Generate sample summary report

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
       This summary can be used to check the expected samples against those acquired (for example, sample numbers, sample type, samples missing from acquisition or lacking metadata information).
    </font>
</div>

In [ ]:
nPYc.reports.generateReport(dataset, 'sample summary')

### Generate feature summary report

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        The feature summary report provides visualisations summarising the quality of the dataset and highlighting any problematic areas, including:
        <ul>
        <li>The distribution of feature intensities in each sample class.</li>
        <li>The TIC in each sample against sample acqusition order, coloured by both sample class, and instrument gain parameter (detector voltage). This provides insight into potential run-order and batch effects.</li>
        <li>The correlation of feature intensity to dilution and TIC in the dilution series. This provides insight into potential issues in correlation to dilution.</li>
        <li>A histogram of feature RSDs, and a plot comparing the RSD measured in the different sample classes (study reference sample, study sanples etc). This provides insight into variance structures in the dataset, with the expectation that biologcal variance should exceeed analytical variance.</li>
        <li>An ion map visualises the location of the detected features in the m/z and retention time space of the assay.</li>
        </ul>
    </font>
</div>

In [ ]:
nPYc.reports.generateReport(dataset,'feature summary')

### Assess batch and run-order effects and apply correction if necessary

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        Visualise feature intensity vs analysis order for a small subset of features in order to assess the need to correct batch and run-order effects. 
    </font>
</div>

In [ ]:
nPYc.reports.generateReport(dataset, 'batch correction assessment')

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
       Apply correction if necessary.
    </font>
</div>

In [ ]:
datasetCorrected = nPYc.batchAndROCorrection.correctMSdataset(dataset, window=11)

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        Compare the dataset before and after correction.
    </font>
</div>

In [ ]:
nPYc.reports.generateReport(dataset, 'batch correction summary', msDataCorrected=datasetCorrected)

### Filter features

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        Generate the feature selection report to assess the number of features passing the quality control thresholds described in: <br/>
        &emsp; Lewis et al, Development and Application of Ultra-Performance Liquid Chromatography-TOF MS for Precision Large Scale <br/> &emsp; Urinary Metabolic Phenotyping, Anal. Chem., 2016, 88 (18), pp 9004–9013
        <br/><br/>
        Filter the features based on these parameters using the 'updateMasks' function. To keep all samples, set the 'filterSamples' argument to False.
    </font>
</div>

In [ ]:
nPYc.reports.generateReport(datasetCorrected,'feature selection')

In [ ]:
datasetCorrected.updateMasks(filterFeatures=True, filterSamples=False)

### Filter samples

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        Remove samples which fail based on any of the above analytical criteria by applying the sample masks.
        <br/><br/>
        At this point we can also exclude any other samples which are not required by setting preferences with the 'sampleTypes' argument. In this example, we limit our dataset to study samples and quality control samples only.
        <br/><br/>
        To keep all features, set the 'filterFeatures' argument to False.
    </font>
</div>

In [ ]:
datasetCorrected.updateMasks(sampleTypes=[SampleType.StudySample, SampleType.StudyPool], filterSamples=True, filterFeatures=False)

### Permanently exclude masked samples/features

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
If happy with the samples and features masked for exclusion, apply these exclusions (permanently remove samples/features from the dataset) using the 'applyMasks' function.
    </font>
</div>

In [ ]:
datasetCorrected.applyMasks()

# 4. Analytical Multivariate Quality Control

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        The analytical multivariate report provides visualisations summarising the largest sources of variance in the dataset (by PCA) with particular emphasis on any potential analytical sources. Plots include:
        <ul>
        <li>Model statistics.</li>
        <li>Scores plots. This provides insight into the relationship between sample, for example, consistency of the QC samples, sample outliers etc. </li>
        <li>Loadings plots. This provides insight into the features with the largest variance in the dataset.</li>
        <li>Potential associations with analytical parameters. Correlation (for continuous metadata) or Kruskal-Wallis test (for categorical metadata) between each metadata field and each set of PCA scores generated, any significant associations are flagged.</li>
        <li>The default scaling is unit variance ('scaling=1'), but other scaling options are available (0 for means centering; 0.5 for Pareto scaling)</li>
        </ul>
    </font>
</div>

In [ ]:
PCAmodelAnalytical = nPYc.multivariate.exploratoryAnalysisPCA(datasetCorrected, scaling=1)

In [ ]:
nPYc.reports.multivariateQCreport(datasetCorrected, PCAmodelAnalytical)

### OPTIONAL: generate interactive scores and loadings plots

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        Interactive scores plot:
        <br/><br/>
        For example, plot the scores for PCA components 1 vs. 2 and colour by Class.
    </font>
</div>

In [ ]:
data = nPYc.plotting.plotScoresInteractive(datasetCorrected, PCAmodelAnalytical, 'Class', components=[1, 2])
iplot(data)

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
        Interactive loadings plot:
        <br/><br/>
        For example, plot the loadings for PCA component 2.
    </font>
</div>

In [ ]:
data = nPYc.plotting.plotLoadingsInteractive(datasetCorrected, PCAmodelAnalytical, component=2)
iplot(data)

# 5. Finalise & Export Dataset

### Check final dataset output:

In [ ]:
nPYc.reports.generateReport(datasetCorrected, 'final report', pcaModel=PCAmodelAnalytical)

### Export data:

<div style='background-color:#EEFFEC'>
    <font color='#0B6D01'>
    Export a unified csv file, which contains the intensity data (one row per sample, one column per variable), alongside all sample and feature metadata (left columns and top rows respectively).
    <br/><br/>
    Output the final report to provide a summary of the dataset.
    </font>
</div>

In [ ]:
if not os.path.exists(saveDir):
    os.makedirs(saveDir)

In [ ]:
datasetCorrected.exportDataset(saveFormat='UnifiedCSV', destinationPath=saveDir)

In [ ]:
nPYc.reports.generateReport(datasetCorrected, 'final report', pcaModel=PCAmodelAnalytical, destinationPath=saveDir)